In [1]:
import requests
import json
import datetime

In [2]:
# Définir la date de début et de fin pour le mois d'octobre
import requests
from datetime import datetime, timedelta

def import_url(page, start_date, end_date):
    # URL de l'API avec les paramètres de date
    url = f"https://www.bandsintown.com/choose-dates/fetch-next/upcomingEvents?date=&page={page}&longitude=-74.006&latitude=40.7128&genre_query=all-genres&start_date={start_date}&end_date={end_date}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }


    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Vérifiez si la requête a réussi
    data = response.json()  # Essayez de décoder la réponse JSON

    return data
data = import_url(1, "2024-10-01", "2024-10-02")
data['events'][0]['artistImageSrc'] == data['events'][0]['artistImageSrc']

True

In [4]:
import requests
from time import sleep
import random

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

liste_dictionnaire = []
artist_name_0 = ""


def recuperer_page():
    d_data = dict()
    max_day = 6
    min_day = 1
    for day in range(min_day, max_day):
        for page in range(1,100):
            # Temps d'attente aléatoire pour simuler un humain
            sleep(random.uniform(0.5, 1.5)) # aléatoire

            # On choppe les données
            data_page = import_url(page, f"2024-10-{day}", f"2024-10-{day+1}")
            data_page_next = import_url(page+1, f"2024-10-{day}", f"2024-10-{day+1}")
            
            # Comparaison des données des deux pages
            if data_page['events'][0]['artistImageSrc'] != data_page_next['events'][0]['artistImageSrc']:
                d_data['day'] = data_page['events']
            else: break
        if max_day > 30:
            max_day = 32
            min_day = 31
        else:
            max_day +=5
            min_day+=5
    return d_data

recuperer_page()

            
            


HTTPError: 403 Client Error: Forbidden for url: https://www.bandsintown.com/choose-dates/fetch-next/upcomingEvents?date=&page=13&longitude=-74.006&latitude=40.7128&genre_query=all-genres&start_date=2024-10-3&end_date=2024-10-4